In [1]:
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings

In [2]:
embeddings = HuggingFaceEmbeddings(
    model_name="BAAI/bge-large-en-v1.5",            # Provide the pre-trained model's path
    model_kwargs={'device':'mps'},                  # Pass the model configuration options
    encode_kwargs={'normalize_embeddings': True}    # Pass the encoding options
)

/Users/siddhantsmedar/miniforge3/envs/rag_project/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/siddhantsmedar/miniforge3/envs/rag_project/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [3]:
db = Chroma(persist_directory="./human_nutrition_vectorstore", embedding_function=embeddings)

In [5]:
query = "reduce the risk of developing diet-related chronic  disease."

In [6]:
retriever = db.as_retriever(search_type="mmr",search_kwargs={"k": 1})
context = retriever.get_relevant_documents(query)

/Users/siddhantsmedar/miniforge3/envs/rag_project/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [7]:
from langchain.prompts import PromptTemplate

# Define your prompt template with placeholders for context and question
template = PromptTemplate(
    input_variables=["context", "question"],
    template="You are a super expert in human nutrition, answer the question using just context provided and if you don't know the answer don't try to make one, considering the following context: {context}\nHuman: {question}\nAssistant:"
)

# Summarize retrieved documents for conciseness
prompt = template.format(context=context, question=query)

In [8]:
from langchain_community.llms import Ollama

llm = Ollama(model="llama3:instruct",temperature=0.2)

llm.invoke(prompt)

"Based on the provided context, it seems that you're referring to functional foods that can help reduce the risk of developing diet-related chronic diseases.\n\nAccording to the Academy of Nutrition and Dietetics (AND), functional foods are reported to not only meet basic nutritional needs but also fight illness and aging. The AND recognizes four types of functional foods: conventional, novel, improved, and qualified.\n\nIn this case, it appears that you're looking for information on how functional foods can help reduce the risk of developing diet-related chronic diseases."